# Fundamental Frequency Tracking Tester

You can feel free to use this file, or you can make a python file to test things, or even put your tests at the bottom of freq.py.  In the case that you use .py files instead of this notebook, save the audio results with the method save_audio in audiotools.py


In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import librosa
import matplotlib.pyplot as plt
from audiotools import *
from effects import *
from freq import *
import IPython.display as ipd

### Compute and plot YIN frequency observations on a file

In [ ]:
filename = "cher.wav"
x_audio, sr = load_audio(filename)

frame_length = 4096
win_length = frame_length//2
hop_length = frame_length//4

all_freqs = get_yin_freqs(x_audio, frame_length, sr)


X = []
for t, freqs_t in enumerate(all_freqs):
    for (f, thresh) in freqs_t:
        X.append([t, f, thresh])
X = np.array(X)
plt.figure(figsize=(15, 6))
plt.scatter(X[:, 0]*hop_length/sr, X[:, 1], s=10, c=X[:, 2], cmap='magma')
plt.colorbar()
plt.xlabel("Time (Seconds)")
plt.ylabel("Frequency (hz)")
plt.title(filename + " Yin Frequency Estimates")

plt.savefig("{}.png".format(filename[0:-4]), bbox_inches='tight')

### Compute and sonify the results of naive "greedy observation" fundamental frequency estimation

In [ ]:
freqs = naive_freqs(all_freqs)
ipd.Audio(sonify_pure_tone(x_audio, freqs, frame_length//4, sr), rate=sr)

### Compute and sonify the results of HMM-based "pyin" fundamental frequency estimation

In [ ]:
freqs = pyin_freqs(all_freqs)
ipd.Audio(sonify_pure_tone(x_audio, freqs, frame_length//4, sr), rate=sr)

### Do a wiggly autotune example on Cher

In [ ]:
allowed_notes = get_scale("G Flat", major=True)
notes, y_auto = autotune(x_audio, freqs, hop_length, allowed_notes, wiggle_amt=1)
plt.figure()
plt.plot(notes)
ipd.Audio(y_auto, rate=sr)

### Do FM synthesis to make Cher sound like a trumpet

In [ ]:
y_fm = fm_synth(x_audio, freqs, frame_length//4, sr, ratio=1, I=16)
ipd.Audio(y_fm, rate=sr)